# Structure Function science showcase

In [ ]:
import tape
import eztao

import numpy as np
import matplotlib.pyplot as plt


In [ ]:
from eztao.carma import DRW_term
from eztao.ts import gpSimRand

# amp is RMS amplitude and tau is the decorrelation timescale
amp = 0.2
tau = 100

# Create the kernel
DRW_kernel = DRW_term(np.log(amp), np.log(tau))

num_light_curves = 100
snr = 10
duration_in_days = 365 * 10
num_observations = 200

# Generate `num_light_curves` lightcurves
# t, y, yerr are np.ndarray with shape = (num_light_curves, num_observations)
t, y, err = gpSimRand(
    carmaTerm=DRW_kernel,
    SNR=snr,
    duration=duration_in_days,
    N=num_observations,
    nLC=num_light_curves)

# pick 10 lightcurves at random and plot
fig, ax = plt.subplots(10, 1, dpi=150, figsize=(8,16))
# Provide some default padding between the 10 lightcurve subplots
fig.tight_layout()
for i, j in enumerate(np.random.choice(len(t), size=10, replace=False)):
    ax[i].errorbar(t[j], y[j], err[j], fmt='.')
    ax[i].set_ylim(-0.5, 0.5)

Run the example through the basic SF and plot

In [ ]:

res_basic = tape.analysis.calc_sf2(t[0], y[0], err[0])

plt.figure()
plt.plot(res_basic['dt'], res_basic['sf2'])
plt.yscale('log')
plt.xscale('log')
plt.title("Basic Structure Function")
plt.show()

## Run the data through the other SF methods and plot those.

In [ ]:
arg_container = tape.analysis.structure_function.StructureFunctionArgumentContainer()
arg_container.sf_method = 'macleod_2012'
res_macleod = tape.analysis.calc_sf2(t[0], y[0], err[0], argument_container=arg_container)

arg_container = tape.analysis.structure_function.StructureFunctionArgumentContainer()
arg_container.sf_method = 'bauer_2009a'
res_bauer_a = tape.analysis.calc_sf2(t[0], y[0], err[0], argument_container=arg_container)

arg_container = tape.analysis.structure_function.StructureFunctionArgumentContainer()
arg_container.sf_method = 'bauer_2009b'
res_bauer_b = tape.analysis.calc_sf2(t[0], y[0], err[0], argument_container=arg_container)

arg_container = tape.analysis.structure_function.StructureFunctionArgumentContainer()
arg_container.sf_method = 'schmidt_2010'
res_schmidt = tape.analysis.calc_sf2(t[0], y[0], err[0], argument_container=arg_container)


In [ ]:
plt.plot(res_basic['dt'], res_basic['sf2'], 'b', label='Basic')
plt.plot(res_macleod['dt'], res_macleod['sf2'], 'g', label='MacLeod 2012')
plt.plot(res_bauer_a['dt'], res_bauer_a['sf2'], 'r', label='Bauer 2009a')
plt.plot(res_bauer_b['dt'], res_bauer_b['sf2'], 'k', label='Bauer 2009b')
plt.plot(res_schmidt['dt'], res_schmidt['sf2'], 'm', label='Schmidt 2010')
plt.legend()
plt.yscale('log')
plt.xscale('log')

## Working with an ensemble

In [ ]:

data_dict = dict(
    lc_id = np.repeat(np.arange(1,101, dtype=int), repeats=200),
    band = np.hstack(np.full(np.shape(t), 'r')),
    time=np.hstack(t),
    flux=np.hstack(y),
    err=np.hstack(err)
)

In [ ]:
from tape.analysis import calc_sf2
manual_colmap = tape.ColumnMapper().assign(
    id_col='lc_id', time_col='time', flux_col='flux', err_col='err', band_col='band',
)
ens = tape.Ensemble()

ens.from_source_dict(source_dict=data_dict, column_mapper=manual_colmap)
batch_res = ens.batch(calc_sf2, compute=True)
batch_res

In [ ]:
plt.figure()
plt.plot(batch_res['dt'], batch_res['sf2'], '.')
plt.yscale('log')
plt.xscale('log')
plt.title("Basic Structure Function")
plt.show()

In [ ]:
# Running the `calc_sf2` function with the first 10 lightcurves in `combine` mode.
arg_container = tape.analysis.structure_function.StructureFunctionArgumentContainer()
arg_container.combine = True
res_basic = tape.analysis.calc_sf2(data_dict['time'][0:2000], data_dict['flux'][0:2000], data_dict['err'][0:2000], argument_container=arg_container)

plt.figure()
plt.plot(res_basic['dt'], res_basic['sf2'])
plt.yscale('log')
plt.xscale('log')
plt.title("Basic Structure Function")
plt.show()